# Customizing Run Names

Every LangSmith run receives a name. This name is visible in the UI and can be employed later for querying a particular run. In the context of tracing chains constructed with LangChain, the default run name is derived from the class name of the invoked object.

For runs categorized as "Chain", the name can be configured by calling the [runnable](https://python.langchain.com/docs/expression_language/) object's `with_config({"run_name": "My Run Name"})` method. This guide illustrates its application through several examples.

**Note:** Only chains and general runnables support custom naming; LLMs, chat models, prompts, and retrievers do not.

Begin by installing the latest version of LangChain.


In [ ]:
# %pip install -U langchain --quiet

In [ ]:
from langsmith import Client

client = Client()

## Example 1: Simple Chain

In [50]:
from langchain import chat_models, prompts, callbacks, schema

chain = (
    prompts.ChatPromptTemplate.from_template("Reverse the following string: {text}")
    | chat_models.ChatOpenAI()
).with_config({"run_name": "StringReverse"})


with callbacks.collect_runs() as cb:
    for chunk in chain.stream({"text": "🌑🌒🌓🌔🌕"}):
        print(chunk.content, flush=True, end="")
    run_id = cb.traced_runs[0].id

🌕🌔🌓🌒🌑

This will result in a trace that looks something like the following:

[![StringReverse](https://smith.langchain.com/public/6fffe5a8-2561-4a73-bb7a-e40bc966ba33/r/img/moon_reverse.png)](https://smith.langchain.com/public/6fffe5a8-2561-4a73-bb7a-e40bc966ba33/r)

If you inspect the run object, you can see the run name is now "StringReverse". You can query within a project for runs with this name to see all the times this chain was used. Do so using the filter syntax `eq(name, "MyRunName")`.

In [ ]:
print(f"Saved name: {client.read_run(run_id=run_id).name}")

Saved name: StringReverse


In [62]:
# List with the name filter to get runs with the assigned name
next(client.list_runs(project_name="default", filter='eq(name, "StringReverse")'))

Run(id=UUID('8e8c455e-6c35-494a-837e-35e330b400bc'), name='StringReverse', start_time=datetime.datetime(2023, 9, 6, 18, 39, 32, 866661), run_type='chain', end_time=datetime.datetime(2023, 9, 6, 18, 39, 34, 423740), extra={'runtime': {'cpu': {'time': {'sys': 3.7803776, 'user': 6.692102656}, 'percent': 0.0, 'ctx_switches': {'voluntary': 82171.0, 'involuntary': 0.0}}, 'mem': {'rss': 60358656.0}, 'library': 'langchain', 'runtime': 'python', 'platform': 'macOS-13.4.1-arm64-arm-64bit', 'sdk_version': '0.0.33', 'thread_count': 46.0, 'library_version': '0.0.283', 'runtime_version': '3.11.2', 'langchain_version': '0.0.283', 'py_implementation': 'CPython'}}, error=None, serialized=None, events=[{'name': 'start', 'time': '2023-09-06T18:39:32.866661'}, {'name': 'end', 'time': '2023-09-06T18:39:34.423740'}], inputs={'text': '🌑🌒🌓🌔🌕'}, outputs={'output': {'content': '🌘🌗🌖🌕🌒', 'example': False, 'additional_kwargs': {}}}, reference_example_id=None, parent_run_id=UUID('e2723ec0-6129-40ed-8460-7ca37248155

## Example 2: Runnable Lambda

LangChain's [RunnableLambdas](https://api.python.langchain.com/en/latest/schema/langchain.schema.runnable.base.RunnableLambda.html#langchain.schema.runnable.base.RunnableLambda) are custom functions that can be invoked, batched, streamed, and/or transformed.

By default (in langchain versions >= 0.0.283), the name of the lambda is the function name. You can customize this by calling `with_config({"run_name": "My Run Name"})` on the runnable lambda object.

In [52]:
def reverse_and_concat(txt: str) -> str:
    return txt[::-1] + txt

lambda_chain = chain | schema.output_parser.StrOutputParser() | reverse_and_concat

In [53]:
with callbacks.collect_runs() as cb:
    print(lambda_chain.invoke({"text": "🌑🌒🌓🌔🌕"}))
    # We will fetch just the lambda run (which is the last child run in this root trace)
    run_id = cb.traced_runs[0].child_runs[-1].id

🌑🌒🌓🌔🌕🌕🌔🌓🌒🌑


In [54]:
# If you are using LangChain < 0.0.283, this will be "RunnableLambda"
print(f"Saved name: {client.read_run(run_id=run_id).name}")

Saved name: reverse_and_concat


The lambda function's trace will be given the lambda function's name, `reverse_and_concat`, as shown below:

<a href="https://smith.langchain.com/public/86ff7b17-4a84-4f99-8a98-13c49658af89/r/54ea562c-d30a-4d71-8c98-5c174819c9dd" target="_blank"><img src="img/reverse_and_concat.png" alt="reverse_and_concat" width="75%"></a>

#### Customize Lambda Name

In the `lambda_chain` above, our function was automatically promoted to a "RunnableLambda" via the piping syntax.
We can customize the run name using the `with_config` syntax once the object is created.

In [55]:
from langchain.schema import runnable

configured_lambda_chain = chain | schema.output_parser.StrOutputParser() | runnable.RunnableLambda(reverse_and_concat).with_config({"run_name": "LambdaReverse"})

In [56]:
with callbacks.collect_runs() as cb:
    print(configured_lambda_chain.invoke({"text": "🌑🌒🌓🌔🌕"}))
    run_id = cb.traced_runs[0].child_runs[-1].id

🌒🌕🌖🌗🌘🌘🌗🌖🌕🌒


In [58]:
print(f"Saved name: {client.read_run(run_id=run_id).name}")

Saved name: LambdaReverse


[![LambdaReverse](img/custom_lambda.png)](https://smith.langchain.com/public/43eea194-b0f0-452b-b92c-4ae39ffd0799/r)

The lambda function's name now will be `LambdaReverse`, as shown below:


## Example 3: Agents

Since LangChain agents and agent executors are types of chains.

In [41]:
from typing import Optional
import requests

from langchain import agents, tools


agent_executor = agents.initialize_agent(
    llm=chat_models.ChatOpenAI(),
    tools=[tools.ReadFileTool(), tools.WriteFileTool(), tools.ListDirectoryTool()],
    agent=agents.AgentType.OPENAI_FUNCTIONS,
)

In [45]:
with callbacks.collect_runs() as cb:
    result = agent_executor.with_config({"run_name": "File Agent"}).invoke("What files are in the current directory?")
    run_id = cb.traced_runs[0].id
    print(result['output'])

The current directory contains the following files:

1. run-naming.ipynb
2. img
3. .ipynb_checkpoints


In [46]:
print(f"Saved name: {client.read_run(run_id=run_id).name}")

Saved name: File Agent


The resulting agent trace will reflect the custom name you've assigned to it.

<a href="https://smith.langchain.com/public/00537050-0da5-4f95-ba28-857183ae9b0c/r" target="_blank"><img src="img/file_agent.png" alt="File Agent Trace" width="75%"></a>

## Conclusion

An easy way to customize run names is to use the `with_config` syntax on your LangChain chain or runnable lambda. This makes it easier to understand a trace at a glance.